# Resampling



## 0.환경준비

### 0.1 라이브러리 로딩

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### 0.2 데이터 로딩

> 예제 데이터는 반도체 공정간 불량을 예측하는 데이터입니다.  
여기서는 class imbalance 에 대한 샘플링과 성능에만 초점을 맞추겠습니다.


In [ ]:
# data data
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/Attrition2.csv"
data = pd.read_csv(path)

## 1.데이터 탐색


* 데이터의 크기를 살펴봅시다.

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
target = 'Attrition'

In [ ]:
print(data[target].value_counts())

data[target].value_counts().plot(kind = 'barh')
plt.show()

## 2.데이터 준비


### 2.1 x, y로 나누기 

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:,target]

### 2.2 가변수화

In [ ]:
dummy_vars = ['Education','Department','EducationField','Gender','JobRole','JobSatisfaction','MaritalStatus',
              'RelationshipSatisfaction','WorkLifeBalance'] # 가변수화 대상 변수 이름 리스트 만든다
x = pd.get_dummies(x, columns = dummy_vars, drop_first=True) 

### 2.3 train : validation 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 2022)

In [ ]:
x_train.shape

## 3.모델링 with Resampling

In [ ]:
# 필요한 라이브러리, 함수들을 불러옵시다.
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

### 3.1 기본 모델링

* 로지스틱 회귀 + 데이터 그대로 이용

In [ ]:
model = LogisticRegression()
model.fit(x_train, y_train)
pred = model.predict(x_val)

print(confusion_matrix(y_val, pred))
print('-' * 55)
print(classification_report(y_val, pred))

LogisticRegression : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html  
* 이전 버전 solver = 'liblinear'
* 최근 버전 solver = 'lbfgs' (max_iter 옵션 경고 문구들이 발생)

In [ ]:
model = LogisticRegression(solver = 'liblinear')
model.fit(x_train, y_train)
pred = model.predict(x_val)

print(confusion_matrix(y_val, pred))
print('-' * 55)
print(classification_report(y_val, pred))

### 3.2 모델링 with Resampling

* Down Sampling + 로지스틱 회귀
    * Resamlpling
    * 모델링
    * 평가 : confusion_matrix, classification_report

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
# down sampling
down_sample = RandomUnderSampler()
d_x_train, d_y_train = down_sample.fit_resample(x_train, y_train)

print(np.bincount(d_y_train))
print(np.bincount(d_y_train) / d_y_train.shape[0])

model_d = LogisticRegression(solver = 'liblinear')
model_d.fit(d_x_train, d_y_train)
pred_d = model_d.predict(x_val)

print(f1_score(y_val,pred_d, pos_label=1))
print('-' * 55)
print(confusion_matrix(y_val, pred_d))
print('-' * 55)
print(classification_report(y_val, pred_d))

* Up Sampling + 로지스틱 회귀
    * Resamlpling
    * 모델링
    * 평가 : confusion_matrix, classification_report

In [ ]:
# up sampling
up_sample = RandomOverSampler()
u_x_train, u_y_train = up_sample.fit_resample(x_train, y_train)

print(np.bincount(u_y_train))
print(np.bincount(u_y_train) / u_y_train.shape[0])

model_u = LogisticRegression(solver = 'liblinear')
model_u.fit(u_x_train, u_y_train)
pred_u = model_u.predict(x_val)

print(f1_score(y_val,pred_u, pos_label=1))
print('-' * 55)
print(confusion_matrix(y_val, pred_u))
print('-' * 55)
print(classification_report(y_val, pred_u))

* SMOTE Sampling + 로지스틱 회귀
    * Resamlpling
    * 모델링
    * 평가 : confusion_matrix, classification_report

In [ ]:
# SMOTE
smote = SMOTE()
s_x_train, s_y_train = smote.fit_resample(x_train, y_train)

print(np.bincount(s_y_train))
print(np.bincount(s_y_train) / s_y_train.shape[0])

model_s = LogisticRegression(solver = 'liblinear')
model_s.fit(s_x_train, s_y_train)
pred_s = model_u.predict(x_val)

print(f1_score(y_val,pred_s, pos_label=1))
print('-' * 55)
print(confusion_matrix(y_val, pred_s))
print('-' * 55)
print(classification_report(y_val, pred_s))

### 3.3 각 resampling 기법별로 성능을 비교해 봅시다.
여러번 반복실행한 다음 f1 score 성능 비교


In [ ]:
def lr_modeling(x_train, y_train, x_val, y_val) :
    m = LogisticRegression(solver = 'liblinear')
    m.fit(x_train, y_train)
    pred = m.predict(x_val)
    return f1_score(y_val,pred, pos_label=1)

In [ ]:
result_d, result_u, result_s = [], [], []

for i in range(50):
    x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .4)
    d_x_train, d_y_train = down_sample.fit_resample(x_train, y_train)
    u_x_train, u_y_train = up_sample.fit_resample(x_train, y_train)
    s_x_train, s_y_train = smote.fit_resample(x_train, y_train)

    result_d.append(lr_modeling(d_x_train, d_y_train, x_val, y_val))
    result_u.append(lr_modeling(u_x_train, u_y_train, x_val, y_val))
    result_s.append(lr_modeling(s_x_train, s_y_train, x_val, y_val))
    print(i)

In [ ]:
# 그래프로 비교
plt.figure(figsize=(12, 8))
sns.kdeplot(result_d, label = 'down')
sns.kdeplot(result_u, label = 'up')
sns.kdeplot(result_s, label = 'smote' )

plt.title('F1 Score')
plt.legend()
plt.grid()
plt.show()


## 4.실습 : Semiconductor manufacturing process dataset


![](https://assets.pandaily.com/uploads/2021/10/semiconductor.png)

* 반도체 제조 공정은 시점별로 수많은 센서로부터 정보를 수집하여 공정을 감시합니다. 
* 센서정보와 함께 공정간 발생된 불량품에 대한 정보를 저장하였습니다.
* 불량을 예측해 봅시다.

### 4.1 데이터 준비

* 데이터 로딩

In [ ]:
path = "https://raw.githubusercontent.com/DA4BAM/dataset/master/secom_9.csv"
data = pd.read_csv(path)

data['label'] = 0
data.loc[data['defeat']== 'defeat', 'label']= 1
data.drop(['datetime','defeat'], axis = 1, inplace=True)
data.head()

변수 정보 
* label : 1 - 불량, 0 - 정상
* v### : 센서값들


In [ ]:
target = 'label'

In [ ]:
data[target].value_counts() / data.shape[0]

* x, y로 나누기 

In [ ]:
x = data.drop(target, axis = 1)
y = data.loc[:, target]

* 가변수화 Dummy Variable  
모두 숫자 데이터이므로 가변수화 대상은 없습니다.


* 데이터 분할
    * 이미 test set은 분할되어 있다고 가정합니다.
    * 주어진 데이터를 train set : validation set 으로 분할

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# train_val에서 train : val = 8 : 2
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state = 2022)

In [ ]:
print(x_train.shape, x_val.shape)

### 4.2 기본모델링
> * 알고리즘은 로지스틱 회귀를 이용합시다.
* 데이터셋을 그대로 모델링을 수행하시오.
* validate set으로 예측하고 평가(classification report, f1 score)해 봅시다.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import * 

### 4.3 모델링 with Resampling
> * down, up, smote 샘플링후 각각 데이터를 이용하여 모델링 하시오.
* 알고리즘 : 기존 샘플과 비교를 위해서 기본모델링에 사용한 알고리즘 사용
* validate set으로 예측하고 평가(classification report, f1 score)해 봅시다.


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
# down sampling


In [ ]:
# up sampling


In [ ]:
# SMOTE


* 모델링

* 로지스틱 회귀- 여러번 반복 실행

3.3의 코드를 붙여놓고 실행해 봅시다.